In [ ]:
import pandas as pd
import sys
from os import getcwd
from os.path import join
import seaborn as sns

sys.path.insert(0, join(getcwd(), "../module_code"))
from data.utils import get_pt_type_indicators

data_path = "/home/davina/Private/crrt-data"
top_features = ["CT-INTEM_skew", "CT-EXTEM_skew", "ABSOLUTE NUCLEATED RBC COUNT_skew", "ALPHA-ANGLE-EX_len"]
filters = {
        "heart": lambda df: df["heart_pt_indicator"] == 1,
        "liver": lambda df: df["liver_pt_indicator"] == 1,
        "infection": lambda df: df["infection_pt_indicator"] == 1,
        "no_heart_liver_infection": lambda df: (
            (df["infection_pt_indicator"] == 0)
            & (df["heart_pt_indicator"] == 0)
            & (df["liver_pt_indicator"] == 0)
        ),
    }

# Missingness for the subgroups/filters

We saw a spike in performance in heart patients, we are assuming they got really sick and had measurements, this is to validate that

In [ ]:
gather_count = {}
gather_notna = {}
for i in range(7):
    if int(i):
        file = f"df_[startdate+{i}-7d,startdate+{i}].parquet"
    else:
        file = "df_[startdate-7d,startdate].parquet"

    f_df = pd.read_parquet(join(data_path, file))
    f_df = get_pt_type_indicators(f_df)

    for filter_name in filters.keys():
        gather_count[(i, filter_name)] = f_df[filters[filter_name]][top_features].describe().loc["count"]
        gather_notna[(i, filter_name)] = f_df[filters[filter_name]][top_features].notna().mean()*100

In [ ]:
def plot_overtime(gather_df: pd.DataFrame, sharey: bool = False):
    counts = pd.DataFrame.from_dict(gather_df, orient="index")
    melted = counts.melt(ignore_index=False, var_name="Feature").reset_index().rename({"level_0": "time", "level_1": "filter", "value": "count"}, axis=1)
    g = sns.FacetGrid(melted, col="Feature", col_wrap=4, sharey=sharey)
    # x, y, hue
    g.map(sns.lineplot, "time", "count", "filter")
    g.add_legend()

In [ ]:
plot_overtime(gather_count)

In [ ]:
plot_overtime(gather_notna)

As we slide the window over there are fewer patients, but each patient has more data avialble (by a little). Therefore the count drops but the percent notna increases over the slided windows.